<a href="https://colab.research.google.com/github/ngotancuong/PythonAssignment/blob/main/Baitap01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ngtncng","key":"8fbb08ab7541c85d81961228e57f5ee6"}'}

In [3]:
! mkdir ~/.kaggle


In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets list

ref                                                             title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
akshaydattatraykhare/diabetes-dataset                           Diabetes Dataset                                      9KB  2022-10-06 08:55:25           4049        127  1.0              
whenamancodes/covid-19-coronavirus-pandemic-dataset             COVID -19 Coronavirus Pandemic Dataset               11KB  2022-09-30 04:05:11           3352         98  1.0              
thedevastator/fast-food-restaurants-in-the-united-states        Fast Food Restaurants in the United States            4MB  2022-10-08 17:30:38           1150         31  1.0              
whenamancodes/students-performance-in-exams                 

In [8]:
! kaggle competitions download -c m5-forecasting-accuracy

 74% 34.0M/45.8M [00:00<00:00, 72.7MB/s]
100% 45.8M/45.8M [00:00<00:00, 90.8MB/s]


In [15]:
! unzip  m5-forecasting-accuracy.zip -d "/content/input/m5-forecasting-accuracy"


Archive:  m5-forecasting-accuracy.zip
  inflating: /content/input/m5-forecasting-accuracy/calendar.csv  
  inflating: /content/input/m5-forecasting-accuracy/sales_train_evaluation.csv  
  inflating: /content/input/m5-forecasting-accuracy/sales_train_validation.csv  
  inflating: /content/input/m5-forecasting-accuracy/sample_submission.csv  
  inflating: /content/input/m5-forecasting-accuracy/sell_prices.csv  


In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import time
import math
import datetime


# Import widgets
from ipywidgets import widgets, interactive, interact
import ipywidgets as widgets
from IPython.display import display

from math import log, floor
from sklearn.neighbors import KDTree

import seaborn as sns
sns.set_style('whitegrid')

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad

import scipy
import statsmodels
from scipy import signal

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.stattools import adfuller

import itertools
from itertools import cycle
plt.style.use('seaborn')
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/content/input/m5-forecasting-accuracy/sales_train_evaluation.csv
/content/input/m5-forecasting-accuracy/sales_train_validation.csv
/content/input/m5-forecasting-accuracy/calendar.csv
/content/input/m5-forecasting-accuracy/sell_prices.csv
/content/input/m5-forecasting-accuracy/sample_submission.csv


In [17]:
sell_prices_df = pd.read_csv('/content/input/m5-forecasting-accuracy/sell_prices.csv')
train_sales_df = pd.read_csv('/content/input/m5-forecasting-accuracy/sales_train_validation.csv')
calendar_df = pd.read_csv('/content/input/m5-forecasting-accuracy/calendar.csv')
submission_file = pd.read_csv('/content/input/m5-forecasting-accuracy/sample_submission.csv')

In [18]:
sell_prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int64  
 3   sell_price  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB
